<a href="https://colab.research.google.com/github/MonicaRekhaN/NLP-Work/blob/main/NextWordPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing packages

In [ ]:
!pip install numpy
!pip install tensorflow
!pip install keras
!pip install nltk

In [4]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

In [5]:
path = '/content/1661-0.txt'
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 581533


In [6]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

In [8]:
len(words)

109173

In [9]:
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [13]:
for c,i in unique_word_index.items():
    print(c,i,end="")
    # break

000 01 110 2100 31000 410_s_ 510th 611 7117 8120 912_s_ 1012th 1114 12140 1315 14150 151500 161661 1716a 1817 191846 201858 211869 221870 231878 241883 251884 261887 271888 281890 2919th 301_s_ 312 3220 33200 342001 352002 362019 37220 38221b 39226 4022nd 41249 4225 43250 4426 4526_s_ 4627 47270 4829 492_s_ 502nd 513 5230 5331 5435 553rd 564 5740 584000 59421 604700 614_d_ 624th 634½ 645 6550 66501 67596 686 6960 706221541 7164 726_d_ 737 74700 75750 7677 777_s_ 787th 798 80801 81809 8282 8383 8484 8584116 8685 8787 8888 8989 908_d_ 918_s_ 929 9390 949th 95_affaire 96_american 97_bijou_ 98_can_ 99_carte 100_causes 101_chronicle_ 102_danseuse_ 103_disjecta 104_dénouement_ 105_echo_ 106_employé_ 107_en 108_encyclopædia 109_encyclopædia_ 110_evening 111_fait 112_fiancé_ 113_globe_ 114_incognito_ 115_l 116_locus 117_lone 118_morning 119_métier_ 120_nous 121_née_ 122_omne 123_our_ 124_outré_ 125_pall 126_partie 127_pâté 128_répertoire_ 129_sophy 130_st 131_standard_ 132_star_ 133_the 134_th

In [14]:
WORD_LENGTH = 5
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

['the', 'project', 'gutenberg', 'ebook', 'of']
the


Trying one hot encoding first and improving later

In [15]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [16]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

In [17]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=2, shuffle=True).history

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/2
811/811 [==============================] - 286s 350ms/step - loss: 6.0046 - accuracy: 0.1074 - val_loss: 6.9710 - val_accuracy: 0.1046
Epoch 2/2
811/811 [==============================] - 272s 335ms/step - loss: 5.7680 - accuracy: 0.1468 - val_loss: 7.8692 - val_accuracy: 0.1070


In [18]:
model.save('keras_next_word_model.h5')
pickle.dump(history, open("history.p", "wb"))
model = load_model('keras_next_word_model.h5')
history = pickle.load(open("history.p", "rb"))

In [19]:
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_word_index[word]] = 1
    return x
prepare_input("It is not a lack".lower())

it
is
not
a
lack


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [20]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [21]:
def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

In [22]:
q =  "Your life will never be the same again"
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower())[0:5])
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 5))

correct sentence:  Your life will never be the same again
Sequence:  your life will never be
your
life
will
never
be
next possible words:  ['of', 'in', 'to', 'a', 'i']


In [24]:
q =  "Happy birthday"
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower())[0:1])
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 5))

correct sentence:  Happy birthday
Sequence:  happy
happy
next possible words:  ['and', 'of', 'to', 'in', 'said']


In [26]:
q =  "Your life will"
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower())[0:2])
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 5))

correct sentence:  Your life will
Sequence:  your life
your
life
next possible words:  ['and', 'of', 'but', 'is', 'said']
